# Proof of Concept Testing - Pre-trained Tacotron2

This notebook is intended to show proof of concept for using Coqui-AI to convert ebooks into audiobooks. In order to accomplish this, the notebook will use tools from the epud-parser notebook, as well as the Coqui-AI / TTS engine tools to convert the parsed text into audio files.

**Note:** this notebook is intended for use with the `ttsenv` conda environment.

## Outline of steps

1. Convert .epub file to text string
2. Run text through TTS engine
3. Combine audio files

In [1]:
import ebooklib
import torch
import time
import numpy as np
from ebooklib import epub
from bs4 import BeautifulSoup
from IPython.display import Audio
from scipy.io.wavfile import write
from nltk import tokenize, download
download('punkt')

[nltk_data] Downloading package punkt to /home/paperspace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
# ?torch.hub.load

In [3]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# map_location=torch.device('cpu')
tacotron2 = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tacotron2', model_math='fp16')
tacotron2 = tacotron2.to('cuda')
tacotron2.eval()

Using cache found in /home/paperspace/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub
/home/paperspace/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/common.py:14: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"
/home/paperspace/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub/PyTorch/Classification/ConvNets/image_classification/models/efficientnet.py:18: UserWarning: pytorch_quantization module not found, quantization will not be available
  "pytorch_quantization module not found, quantization will not be available"


Tacotron2(
  (embedding): Embedding(148, 512)
  (encoder): Encoder(
    (convolutions): ModuleList(
      (0): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (2): Sequential(
        (0): ConvNorm(
          (conv): Conv1d(512, 512, kernel_size=(5,), stride=(1,), padding=(2,))
        )
        (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (lstm): LSTM(512, 256, batch_first=True, bidirectional=True)
  )
  (decoder): Decoder(
    (prenet): Prenet(
      (layers): ModuleList(
        (0): LinearNorm(
          (lin

## Step 1 - Epub to text

Text here.

In [4]:
book = epub.read_epub('pg2554.epub')
# book = epub.read_epub('epub-test.epub')

Text here.

In [5]:
for item in book.get_items():
   if item.get_type() == ebooklib.ITEM_DOCUMENT:
       print('==================================')
       print('NAME : ', item.get_name())
       print('----------------------------------')
       print(BeautifulSoup(item.get_content(), "html.parser").text)
       print('==================================')

NAME :  5044171427629287690_2554-h-0.htm.html
----------------------------------




The Project Gutenberg eBook of Crime and Punishment, by Fyodor Dostoevsky
This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before using this eBook.
Title: Crime and Punishment
Author: Fyodor Dostoevsky
Translator: Constance Garnett
Release Date: March, 2001 [eBook #2554]
[Most recently updated: August 6, 2021]
Language: English
Character set encoding: UTF-8
Produced by: John Bickers, Dagny and David Widger
*** START OF THE PROJECT GUTENBERG EBOOK CRIME AND PUNISHMENT ***
CRIME AND PUNISHMENT
By Fyodor Dostoevsky
Translated By Constance Garn






 CHAPTER IV
At that moment the door was softly opened, and a young girl walked into the room, looking timidly about her. Everyone turned towards her with surprise and curiosity. At first sight, Raskolnikov did not recognise her. It was Sofya Semyonovna Marmeladov. He had seen her yesterday for the first time, but at such a moment, in such surroundings and in such a dress, that his memory retained a very different image of her. Now she was a modestly and poorly-dressed young girl, very young, indeed, almost like a child, with a modest and refined manner, with a candid but somewhat frightened-looking face. She was wearing a very plain indoor dress, and had on a shabby old-fashioned hat, but she still carried a parasol. Unexpectedly finding the room full of people, she was not so much embarrassed as completely overwhelmed with shyness, like a little child. She was even about to retreat. “Oh... it’s you!” said Raskolnikov, extremely astonished, and he, too, was confused. He at once re






 CHAPTER V
Raskolnikov walked after him.
“What’s this?” cried Svidrigaïlov turning round, “I thought I said...”
“It means that I am not going to lose sight of you now.”
“What?”
Both stood still and gazed at one another, as though measuring their strength.
“From all your half tipsy stories,” Raskolnikov observed harshly, “I am positive that you have not given up your designs on my sister, but are pursuing them more actively than ever. I have learnt that my sister received a letter this morning. You have hardly been able to sit still all this time.... You may have unearthed a wife on the way, but that means nothing. I should like to make certain myself.”
Raskolnikov could hardly have said himself what he wanted and of what he wished to make certain.
“Upon my word! I’ll call the police!”
“Call away!”
Again they stood for a minute facing each other. At last Svidrigaïlov’s face changed. Having satisfied himself that Raskolnikov was not frightened at his threat, he assumed a mirthful an

## Step 2 - Tacotron2 from TorchHub
Text here.

In [6]:
waveglow = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_waveglow', model_math='fp16')
waveglow = waveglow.remove_weightnorm(waveglow)
waveglow = waveglow.to('cuda')
waveglow.eval()

Using cache found in /home/paperspace/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


WaveGlow(
  (upsample): ConvTranspose1d(80, 80, kernel_size=(1024,), stride=(256,))
  (WN): ModuleList(
    (0): WN(
      (in_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(1,))
        (1): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(2,), dilation=(2,))
        (2): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(4,), dilation=(4,))
        (3): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(8,), dilation=(8,))
        (4): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(16,), dilation=(16,))
        (5): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(32,), dilation=(32,))
        (6): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(64,), dilation=(64,))
        (7): Conv1d(512, 1024, kernel_size=(3,), stride=(1,), padding=(128,), dilation=(128,))
      )
      (res_skip_layers): ModuleList(
        (0): Conv1d(512, 1024, kernel_size=(1,), stride=(1,))
        (1): Conv1d(51

In [7]:
text = "Hello world, I missed you so much."

In [8]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
sequences, lengths = utils.prepare_input_sequence([text])

Using cache found in /home/paperspace/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [9]:
with torch.no_grad():
    mel, _, _ = tacotron2.infer(sequences, lengths)
    audio = waveglow.infer(mel)
audio_numpy = audio[0].data.cpu().numpy()
rate = 22050

In [10]:
write("outputs/audio.wav", rate, audio_numpy)

In [11]:
Audio(audio_numpy, rate=rate)

## Step 3 - Text from epub to .wav file

Text here.

In [12]:
max_characters = 0
total_characters = 0
sample_num = 0

for item in book.get_items():
#    if item.get_type() == ebooklib.ITEM_DOCUMENT:
    if item.get_name() == "5044171427629287690_2554-h-1.htm.html":
        # set up text sample and path
        input_text = BeautifulSoup(item.get_content(), "html.parser").text
        input_text = input_text.replace('—', '')
        text_list_tmp = input_text.split('\n')

        text_list = ['']
        for sample in text_list_tmp:
            text_list += tokenize.sent_tokenize(sample)

        # print(input_text)
        # print(text_list)
        for i in range(0,len(text_list)):
#             print("Index:" + str(i) + "Characters:" + str(len(text_list[i])))
#             print(text_list[i])
        
            if len(text_list[i]) >= max_characters:
                max_characters = len(text_list[i])
            
            total_characters += len(text_list[i])
            sample_num += 1
            
print("Max Characters in Sample:" + str(max_characters))
print("Total Characters:" + str(total_characters))
print("Number of Samples:" + str(sample_num))

Max Characters in Sample:336
Total Characters:4419
Number of Samples:40


In [13]:
utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
sequences, lengths = utils.prepare_input_sequence([text_list[6]])

Using cache found in /home/paperspace/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


In [14]:
with torch.no_grad():
    mel, _, _ = tacotron2.infer(sequences, lengths)
    audio = waveglow.infer(mel)
audio_numpy = audio[0].data.cpu().numpy()
rate = 22050

In [15]:
Audio(audio_numpy, rate=rate)

## Step 4 - Convert paragraph

Text here.

In [16]:
# start = time.time()

# # audio_numpy_combined = np.empty(0,)

# for sample in text_list:
#     if sample:      
#         sample_index = text_list.index(sample)
#         sample_path = "outputs/output"+str(sample_index)+".wav"
        
#         print("Sample to convert:" + "\n" + sample)
#         print("Index of sample:" + str(sample_index))
#         print("Output path:" + str(sample_path))
#         print("Characters:" + str(len(sample)) + "\n\n")
        
#         utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
#         sequences, lengths = utils.prepare_input_sequence([sample])
        
#         with torch.no_grad():
#             mel, _, _ = tacotron2.infer(sequences, lengths)
#             audio = waveglow.infer(mel)
#         audio_numpy = audio[0].data.cpu().numpy()
#         rate = 22050
        
#         write(sample_path, rate, audio_numpy)
        
# #         current_wav = AudioSegment.from_file(sample_path)
# #         output_wav = AudioSegment.from_file("output_combined.wav")
        
# #         output_wav += current_wav
# #         output_wav.export(out_f = "output_combined.wav", format = "wav")
        
# #         print("Shape of audio_numpy:" + str(audio_numpy.shape))
# #         print("Shape of audio_numpy_combined before:" + str(audio_numpy_combined.shape))
        
# #         audio_numpy_combined= np.concatenate([audio_numpy_combined, audio_numpy], axis=0)
# #         print("Shape of audio_numpy_combined after:" + str(audio_numpy_combined.shape))

# # write("output_combined.wav", rate, audio_numpy_combined)
        
# end = time.time()
# print("The time of execution of above program is :", end-start)

Sample to convert:
 TRANSLATOR’S PREFACE
Index of sample:1
Output path:output1.wav
Characters:21




Using cache found in /home/paperspace/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Sample to convert:
A few words about Dostoevsky himself may help the English reader to understand his work.
Index of sample:2
Output path:output2.wav
Characters:88




Using cache found in /home/paperspace/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Sample to convert:
Dostoevsky was the son of a doctor.
Index of sample:3
Output path:output3.wav
Characters:35




...

Sample to convert:
In the words of a Russian critic, who seeks to explain the feeling inspired by Dostoevsky: “He was one of ourselves, a man of our blood and our bone, but one who has suffered and has seen so much more deeply than we have his insight impresses us as wisdom... that wisdom of the heart which we seek that we may learn from it how to live.
Index of sample:38
Output path:output38.wav
Characters:336




Using cache found in /home/paperspace/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


Warning! Reached max decoder steps
Sample to convert:
All his other gifts came to him from nature, this he won for himself and through it he became great.”
Index of sample:39
Output path:output39.wav
Characters:101




Using cache found in /home/paperspace/.cache/torch/hub/NVIDIA_DeepLearningExamples_torchhub


The time of execution of above program is : 185.6446168422699


In [17]:
# Audio("outputs/output7.wav", rate=rate)

## Step 5 - Combine output files

Text here.

In [18]:
from pydub import AudioSegment

In [24]:
start = time.time()

output_wav = AudioSegment.empty()

for i in range(1,40):
    current_wav = AudioSegment.from_file("outputs/output" + str(i) + ".wav")
    output_wav += current_wav
    output_wav.export(out_f = "outputs/output_combined.wav", format = "wav")
    
end = time.time()
print("The time of execution of above program is :", end-start)

Audio("outputs/output_combined.wav", rate=rate)

The time of execution of above program is : 12.226416110992432


In [33]:
Audio("outputs/output2.wav", rate=rate)

In [36]:
output_wav = AudioSegment.empty()

wav_1 = AudioSegment.from_file("outputs/output1.wav")
output_wav = output_wav.append(wav_1, crossfade=0)

wav_2 = AudioSegment.from_file("outputs/output2.wav")
output_wav = output_wav.append(wav_2, crossfade=0)

output_wav.export(out_f = "outputs/output_combined.wav", format = "wav")

Audio("outputs/output_combined.wav", rate=rate)

### Step 6 - Convert whole book

Text here.

In [ ]:
max_characters = 0
total_characters = 0
sample_num = 0

for item in book.get_items():
    if item.get_type() == ebooklib.ITEM_DOCUMENT:
#    if item.get_name() == "5044171427629287690_2554-h-1.htm.html":
        # set up text sample and path
        input_text = BeautifulSoup(item.get_content(), "html.parser").text
        input_text = input_text.replace('—', '')
        text_list_tmp = input_text.split('\n')

        text_list = ['']
        for sample in text_list_tmp:
            text_list += tokenize.sent_tokenize(sample)

        # print(input_text)
        # print(text_list)
        for i in range(0,len(text_list)):
#             print("Index:" + str(i) + "Characters:" + str(len(text_list[i])))
#             print(text_list[i])
        
            if len(text_list[i]) >= max_characters:
                max_characters = len(text_list[i])
            
            total_characters += len(text_list[i])
            sample_num += 1
            
print("Max Characters in Sample:" + str(max_characters))
print("Total Characters:" + str(total_characters))
print("Number of Samples:" + str(sample_num))

In [ ]:
start = time.time()

# audio_numpy_combined = np.empty(0,)

for sample in text_list:
    if sample:      
        sample_index = text_list.index(sample)
        sample_path = "outputs/output"+str(sample_index)+".wav"
        
        print("Sample to convert:" + "\n" + sample)
        print("Index of sample:" + str(sample_index))
        print("Output path:" + str(sample_path))
        print("Characters:" + str(len(sample)) + "\n\n")
        
        utils = torch.hub.load('NVIDIA/DeepLearningExamples:torchhub', 'nvidia_tts_utils')
        sequences, lengths = utils.prepare_input_sequence([sample])
        
        with torch.no_grad():
            mel, _, _ = tacotron2.infer(sequences, lengths)
            audio = waveglow.infer(mel)
        audio_numpy = audio[0].data.cpu().numpy()
        rate = 22050
        
        write(sample_path, rate, audio_numpy)
        
#         current_wav = AudioSegment.from_file(sample_path)
#         output_wav = AudioSegment.from_file("output_combined.wav")
        
#         output_wav += current_wav
#         output_wav.export(out_f = "output_combined.wav", format = "wav")
        
#         print("Shape of audio_numpy:" + str(audio_numpy.shape))
#         print("Shape of audio_numpy_combined before:" + str(audio_numpy_combined.shape))
        
#         audio_numpy_combined= np.concatenate([audio_numpy_combined, audio_numpy], axis=0)
#         print("Shape of audio_numpy_combined after:" + str(audio_numpy_combined.shape))

# write("output_combined.wav", rate, audio_numpy_combined)
        
end = time.time()
print("The time of execution of above program is :", end-start)